# импорты

In [1]:
import boto3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import string
import re
import nltk

from tqdm import trange
from nltk import tokenize
from nltk.corpus import stopwords


from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist
from collections import Counter

my_stopwords = [] #заглушка если придумаем стопслова
nltk.download('stopwords')
nltk.download('wordnet')
russian_stopwords_new = stopwords.words("russian")
russian_stopwords_new.extend(my_stopwords)
not_stopwords = {'не', 'ни'}
russian_stopwords = [word for word in russian_stopwords_new if word not in not_stopwords]

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, f1_score
from matplotlib import pyplot as plt

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dariamishina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/dariamishina/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
import sys
print(sys.path)

['/Users/dariamishina/Documents/HSE/YearProject/mlds23-authorship-identification/ML', '/Users/dariamishina/.conda/envs/untitled/lib/python38.zip', '/Users/dariamishina/.conda/envs/untitled/lib/python3.8', '/Users/dariamishina/.conda/envs/untitled/lib/python3.8/lib-dynload', '', '/Users/dariamishina/.conda/envs/untitled/lib/python3.8/site-packages']


In [3]:
# добавляем в path папку с тем файлом, откуда надо импортировать функции
sys.path.append('/Users/dariamishina/Documents/HSE/YearProject/mlds23-authorship-identification')

In [4]:
from data_preproc.src.utils import preprocess_text1, preprocess_text2, preprocess_text3, preprocess_text4, tokenizing

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dariamishina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/dariamishina/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
session = boto3.session.Session()

In [6]:
s3 = session.client(
    service_name='s3',
    endpoint_url='https://storage.yandexcloud.net',
    aws_access_key_id='YCAJErlaldUmioGbHQSqJ70MR',
    aws_secret_access_key='YCPSba_JgloNYSNWcnKO2CYCEB8PFR1Iwgr2jIUy',
    region_name='ru-cental1'
)

# Загружаем уже расспличенные тексты

In [7]:
BUCKET_NAME = "mlds23-authorship-identification"
BUCKET_DIR = "splitted_data/"
FILENAME = 'splitted_df.csv'
#загружает в локальную директорию, потом отдельно надо считывать
s3.download_file(Filename=FILENAME, Bucket=BUCKET_NAME, Key=BUCKET_DIR + FILENAME)

In [7]:
df = pd.read_csv('splitted_df.csv')

In [8]:
df.head()

,target,text,book
0,author_id_06,"Узкими горными тропинками , от одного дачного ...",raw_data/aleksandr_kuprin_belyj_pudel'.txt
1,author_id_06,свернутый ковер для акробатических упражнений ...,raw_data/aleksandr_kuprin_belyj_pudel'.txt
2,author_id_06,"позабытые . Кроме того , были в шарманке две п...",raw_data/aleksandr_kuprin_belyj_pudel'.txt
3,author_id_06,тайной грусти : — Что поделаешь ? .. Древний о...,raw_data/aleksandr_kuprin_belyj_pudel'.txt
4,author_id_06,", да уж ладно ! Кормила она нас с тобой , Серг...",raw_data/aleksandr_kuprin_belyj_pudel'.txt


#  Выделяем классы метками

In [9]:
enc = LabelEncoder()
df['author_id'] = enc.fit_transform(df['target'])

In [10]:
#посмотрим на распределение классов
df.groupby('author_id').author_id.count().sort_values()

author_id
1     379
7     446
8     627
5     725
3     804
6    1047
4    1295
0    1299
9    2387
2    2706
Name: author_id, dtype: int64

In [11]:
labels = df['target']

# подготовка для TF-IDF

In [12]:
#используем самую "суровую предобработку" с лемматизацией
df['text_proc1'] = df['text'].apply(lambda x: preprocess_text1(x, tostr=True))

In [14]:
df.head()

,target,text,book,author_id,text_proc1
0,author_id_06,"Узкими горными тропинками , от одного дачного ...",raw_data/aleksandr_kuprin_belyj_pudel'.txt,6,узкий горный тропинка дачный поселок пробирать...
1,author_id_06,свернутый ковер для акробатических упражнений ...,raw_data/aleksandr_kuprin_belyj_pudel'.txt,6,свернутый ковер акробатический упражнение прав...
2,author_id_06,"позабытые . Кроме того , были в шарманке две п...",raw_data/aleksandr_kuprin_belyj_pudel'.txt,6,позабывать кроме шарманка предательский труба ...
3,author_id_06,тайной грусти : — Что поделаешь ? .. Древний о...,raw_data/aleksandr_kuprin_belyj_pudel'.txt,6,тайный грусть — поделать древний орган прос...
4,author_id_06,", да уж ладно ! Кормила она нас с тобой , Серг...",raw_data/aleksandr_kuprin_belyj_pudel'.txt,6,ладно кормить сергей сей пора бог давать покор...


# TF-IDF

In [15]:
x_train, x_test, y_train, y_test = train_test_split(df['text_proc1'], df['author_id'], test_size = 0.3, random_state=42,\
                                                    stratify=df['author_id'])

## +wandb

In [16]:
import wandb

In [17]:
wandb.login()

wandb: Currently logged in as: dariamishina2812 (datalabai). Use `wandb login --relogin` to force relogin


True

In [23]:
run = wandb.init(project="authorship_identification", name="tfidf_log", entity="mlds23_ai")

In [24]:
#выглядит не очень, почему-то только один класс, видимо для бинарной классификации только
wandb.sklearn.plot_class_proportions(y_train, y_test, labels)

In [25]:
def tfidf_model(x_train, y_train):
    tf_idf = TfidfVectorizer(stop_words=russian_stopwords, max_features=550, min_df=2)
    
    logit = LogisticRegression(C=1e2, n_jobs=4, solver='sag',
                               random_state=42, verbose=0,
                               multi_class='multinomial',
                               fit_intercept=True, max_iter=400)
   
    tfidf_logit_pipeline = Pipeline([('tf_idf', tf_idf), ('logit', logit)])
    
    tfidf_logit_pipeline.fit(x_train, y_train)
    
    # параметры в WandB
    wandb.config.update({
        'max_features': 550,
        'min_df': 2,
        'C': 1e2,
        'solver': 'sag',
        'n_jobs': 4,
        'random_state': 42,
        'multi_class': 'multinomial',
        'fit_intercept': True,
        'max_iter': 400
    })
    
    return tfidf_logit_pipeline

In [26]:
tfidf_logit_pipeline = tfidf_model(x_train, y_train)

y_pred = tfidf_logit_pipeline.predict(x_test)
y_probas = tfidf_logit_pipeline.predict_proba(x_test)

#вот тут модель классификации из пайплайна
# tfidf_logit_pipeline['logit']

# wandb.sklearn.plot_learning_curve(tfidf_logit_pipeline['logit'], x_train, y_train)
#не будет работать для tf-idf,но для статистики можно попробовать

# не будет работать для логрега, а для деревьев-бустингов будет
# wandb.sklearn.plot_feature_importances(tfidf_logit_pipeline['logit']);

# логируем метрики в WandB
f1 = f1_score(y_test, y_pred, average='weighted')
wandb.log({'f1_score_weighted': f1})

f1 = f1_score(y_test, y_pred, average='micro')
wandb.log({'f1_score_micro': f1})

f1 = f1_score(y_test, y_pred, average='macro')
wandb.log({'f1_score_macro': f1})

#выглядит не очень, почему-то только один класс, видимо для бинарной классификации только
wandb.sklearn.plot_roc(y_test, y_probas, labels)

run.finish()

wandb: WARNING wandb.plots.* functions are deprecated and will be removed in a future release. Please use wandb.plot.* instead.


f1_score_macro,▁
f1_score_micro,▁
f1_score_weighted,▁
f1_score_macro,0.59051
f1_score_micro,0.63642
f1_score_weighted,0.63647


## просто пайплайн

In [15]:
def tfidf_model(x_train, y_train):
    
    tf_idf = TfidfVectorizer(stop_words=russian_stopwords, max_features=550, min_df=2)
    
    logit = LogisticRegression(C=1e2, n_jobs=4, solver='sag', #solver='lbfgs', #проблема с этим солвером
                           random_state=42, verbose=0, 
                           multi_class='multinomial',
                           fit_intercept=True, max_iter=400)
   
    tfidf_logit_pipeline = Pipeline([('tf_idf', tf_idf), 
                                 ('logit', logit)])
    tfidf_logit_pipeline.fit(x_train, y_train)
    return tfidf_logit_pipeline

In [16]:
tfidf_logit_pipeline = tfidf_model(x_train, y_train)

In [17]:
%%time
y_pred = tfidf_logit_pipeline.predict(x_test)

CPU times: user 121 ms, sys: 9.85 ms, total: 130 ms
Wall time: 132 ms


In [25]:
f1_score(y_test, y_pred, average='macro')

0.590514812375762

In [26]:
f1_score(y_test, y_pred, average='micro')

0.6364153627311522

In [27]:
f1_score(y_test, y_pred, average='weighted')

0.6364694242915129

## пиклим и загружаем в s3

In [ ]:
import pickle

with open('tfidf_logit_pipeline.pkl', 'wb') as file:
    pickle.dump((tfidf_logit_pipeline), file)

In [ ]:
#вот так загружаем
# with open('tfidf_logit_pipeline.pkl', 'rb') as file:
#     tfidf_logit_pipeline_test = pickle.load(file)

In [ ]:
DATA_PATH = '/Users/dariamishina/Documents/HSE/YearProject/mlds23-authorship-identification/ML/' #это локальный путь! Куда сохранился мой csv файл
NEW_FILE_NAME = 'tfidf_logit_pipeline.pkl'

In [ ]:
s3.upload_file(f'{DATA_PATH}{NEW_FILE_NAME}', BUCKET_NAME, f'models/{NEW_FILE_NAME}')